# Using Private Key JWT Secret

There are two ways of setting up a Private Key Jwt secret

- Configure ClientCredentials asserion as parameters to AddClient
- Using a ClientAssertionService 

## Configure ClientCredentials asserion as parameters to AddClient

In [ ]:
#!csharp
var authority = "https://demo.duendesoftware.com"; // Set your authority URL here"
var clientId = "m2m.jwt";
var jwk = """
{
    "d":"GmiaucNIzdvsEzGjZjd43SDToy1pz-Ph-shsOUXXh-dsYNGftITGerp8bO1iryXh_zUEo8oDK3r1y4klTonQ6bLsWw4ogjLPmL3yiqsoSjJa1G2Ymh_RY_sFZLLXAcrmpbzdWIAkgkHSZTaliL6g57vA7gxvd8L4s82wgGer_JmURI0ECbaCg98JVS0Srtf9GeTRHoX4foLWKc1Vq6NHthzqRMLZe-aRBNU9IMvXNd7kCcIbHCM3GTD_8cFj135nBPP2HOgC_ZXI1txsEf-djqJj8W5vaM7ViKU28IDv1gZGH3CatoysYx6jv1XJVvb2PH8RbFKbJmeyUm3Wvo-rgQ",
    "dp":"YNjVBTCIwZD65WCht5ve06vnBLP_Po1NtL_4lkholmPzJ5jbLYBU8f5foNp8DVJBdFQW7wcLmx85-NC5Pl1ZeyA-Ecbw4fDraa5Z4wUKlF0LT6VV79rfOF19y8kwf6MigyrDqMLcH_CRnRGg5NfDsijlZXffINGuxg6wWzhiqqE",
    "dq":"LfMDQbvTFNngkZjKkN2CBh5_MBG6Yrmfy4kWA8IC2HQqID5FtreiY2MTAwoDcoINfh3S5CItpuq94tlB2t-VUv8wunhbngHiB5xUprwGAAnwJ3DL39D2m43i_3YP-UO1TgZQUAOh7Jrd4foatpatTvBtY3F1DrCrUKE5Kkn770M",
    "e":"AQAB",
    "kid":"ZzAjSnraU3bkWGnnAqLapYGpTyNfLbjbzgAPbbW2GEA",
    "kty":"RSA",
    "n":"wWwQFtSzeRjjerpEM5Rmqz_DsNaZ9S1Bw6UbZkDLowuuTCjBWUax0vBMMxdy6XjEEK4Oq9lKMvx9JzjmeJf1knoqSNrox3Ka0rnxXpNAz6sATvme8p9mTXyp0cX4lF4U2J54xa2_S9NF5QWvpXvBeC4GAJx7QaSw4zrUkrc6XyaAiFnLhQEwKJCwUw4NOqIuYvYp_IXhw-5Ti_icDlZS-282PcccnBeOcX7vc21pozibIdmZJKqXNsL1Ibx5Nkx1F1jLnekJAmdaACDjYRLL_6n3W4wUp19UvzB1lGtXcJKLLkqB6YDiZNu16OSiSprfmrRXvYmvD8m6Fnl5aetgKw",
    "p":"7enorp9Pm9XSHaCvQyENcvdU99WCPbnp8vc0KnY_0g9UdX4ZDH07JwKu6DQEwfmUA1qspC-e_KFWTl3x0-I2eJRnHjLOoLrTjrVSBRhBMGEH5PvtZTTThnIY2LReH-6EhceGvcsJ_MhNDUEZLykiH1OnKhmRuvSdhi8oiETqtPE",
    "q":"0CBLGi_kRPLqI8yfVkpBbA9zkCAshgrWWn9hsq6a7Zl2LcLaLBRUxH0q1jWnXgeJh9o5v8sYGXwhbrmuypw7kJ0uA3OgEzSsNvX5Ay3R9sNel-3Mqm8Me5OfWWvmTEBOci8RwHstdR-7b9ZT13jk-dsZI7OlV_uBja1ny9Nz9ts",
    "qi":"pG6J4dcUDrDndMxa-ee1yG4KjZqqyCQcmPAfqklI2LmnpRIjcK78scclvpboI3JQyg6RCEKVMwAhVtQM6cBcIO3JrHgqeYDblp5wXHjto70HVW6Z8kBruNx1AH9E8LzNvSRL-JVTFzBkJuNgzKQfD0G77tQRgJ-Ri7qu3_9o1M4"
}
""";
var scope = "api";

In [ ]:
#!csharp
#load "../HttpLoggerHelper.csx"
#r "nuget: Duende.AccessTokenManagement, 3.2.0"
#r "nuget: Microsoft.Extensions.Hosting, 9.0.5"
#r "nuget: Microsoft.Extensions.Caching.Memory, 9.0.0"
#r "nuget: Fhi.Authentication.Extensions, 1.0.0"

using Microsoft.Extensions.Hosting;
using Duende.AccessTokenManagement;
using Duende.IdentityModel;
using Duende.IdentityModel.Client;
using Fhi.Authentication.Tokens;
using Microsoft.Extensions.DependencyInjection;
using System.Net.Http;

async Task RunAsync()
{
    var builder = Host.CreateDefaultBuilder();

    builder.ConfigureServices(services =>
    {
        services
            .AddClientCredentialsTokenManagement()
            .AddClient(clientId, options =>
            {
                options.TokenEndpoint = "https://demo.duendesoftware.com/connect/token";
                options.ClientId = clientId;
                options.Parameters = new Duende.IdentityModel.Client.Parameters()
                {
                    { OidcConstants.TokenRequest.ClientAssertionType, OidcConstants.ClientAssertionTypes.JwtBearer },
                    { OidcConstants.TokenRequest.ClientAssertion, ClientAssertionTokenHandler.CreateJwtToken(
                        issuer: authority, 
                        clientId,
                        jwk) 
                    }
                };
                options.Scope = scope;
            });

        services.AddClientCredentialsHttpClient(clientId, clientId, client =>
        {
            client.BaseAddress = new Uri("https://localhost:7150");
        });

        services.AddDistributedMemoryCache();
        services.AddTransient<ITestService, TestService>();
    });

    var host = builder.Build();
    
    using var scope = host.Services.CreateScope();
    var testService = scope.ServiceProvider.GetRequiredService<ITestService>();
    await testService.Get();
}

interface ITestService
{
    public Task Get();
}

// A service that uses the registerd HttpClient with HttpClientFactory to make requests.
class TestService : ITestService
{
    private readonly IHttpClientFactory _factory;
    public TestService(IHttpClientFactory factory)
    {
        _factory = factory;
    }
    
    public async Task Get()
    {
        var client = _factory.CreateClient("m2m.jwt");
        var response = await client.GetAsync("api/v1/integration/health-records");
        await HttpLogger.LogRequest(response.RequestMessage);
        await HttpLogger.LogResponse(response);
    }
}

await RunAsync();